___

# PROJETO 2 - Ciência dos Dados 2021.1
#### Rodrigo Guilherme Araujo Pires

___
<div id="indice"></div>



___

<div id="intro"></div>

## A. Introdução
Este projeto tem o objetivo de prever uma variável principal em função de demais outras variáveis 
que podem influenciar em seu comportamento. Para isso foi escolhida uma base de dados contendo dados de varios filmes lançados desde 2006 e a seguinte pergunta: "Com os dados contidos na base de dados, é possível prever a sua nota?"

___

<div id="dados"></div>

## B. Minerando Dados e Características do Dataset

Importando bibliotecas e dataset:

In [ ]:
%matplotlib inline
import math
import os

import matplotlib.pyplot as plt
import numpy as np
from numpy import arange
import pandas as pd
from scipy import stats

In [ ]:
dados = pd.read_csv('movie_success_rate.csv') # Importa a base de dados
dados=dados.dropna() # Remove linhas com valores faltando

In [ ]:
dados

In [ ]:
# Histograma apenas utilizando o primeiro genero na coluna de generos
primeiro_genero = dados['Genre'].str.split(",", expand=True)[0]
generos = primeiro_genero.value_counts()
primeiro_genero.value_counts()

In [ ]:
plt.figure(figsize=(15, 5))
plt.hist(primeiro_genero, bins=len(generos), edgecolor='white')
plt.title('Número de filmes por gênero')
plt.xlabel('gênero')
plt.ylabel('Número de filmes')
plt.show()

In [ ]:
plt.figure(figsize=(20, 7.5))
plt.title('Relação da renda e e avaliação de cada filme')
plt.scatter(dados['Revenue (Millions)'], dados['Rating'], c='red', alpha=0.5)
plt.xlabel('Receita')
plt.ylabel('Avaliação')
plt.show()

___

<div id="modelos"></div>

## C. Modelos de Predição e Estatísticas de Validação



#### Selecionando as colunas utilizadas para o modelo de predição:

- Para o modelo, serão utilizados dados de ano de lançamento, duração, quantidade de avaliações, receita e os gêneros do filme.


- $y$ será a avaliação do filme

In [ ]:
x = dados[['Year', 'Runtime (Minutes)', 'Votes', 'Revenue (Millions)', 'Action', 'Adventure', 'Aniimation', 'Biography', 'Comedy', 'Crime', 'Drama', 'Family', 'Fantasy', 'History', 'Horror', 'Music', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Sport', 'Thriller', 'War', 'Western']]
x.head(10)

In [ ]:
y = dados['Rating']
y.head(10)

### 1. Random Forest Regression

#### O Random Forest Regression constrói múltiplas árvores de decisão durante seu trinamento, retornando a média das predições de cada árvore. É considerado um modelo robusto e não enviesado, porém requer um maior poder computacional.

Para implementar o modelo será utilizado `RandomForestRegressor` do módulo `sklearn.ensemble`. Outros módulos que serão usados depois também importados.

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

#### Separando a base de dados em treinamento e teste com `train_test_split`:
- parâmetro: `test_size=0.1` - 10% serão usados para teste

In [ ]:
x_treinamento, x_teste, y_treinamento, y_teste = train_test_split(x ,y , test_size=0.1, random_state=0)


#### `RandomForestRegressor` utilizado com os parâmetros:
- `n_estimators` - número de árvores
- `random_state` - controla a aleatoriedade de escolhas do modelo, 0 é a maior consistência
    

In [ ]:
rfr = RandomForestRegressor(n_estimators = 500, random_state=0)
rfr = rfr.fit(x_treinamento, y_treinamento)
y_predicao = rfr.predict(x_teste)

#### Verificando a performance:
- `.score()` (varia de 0 a 1) - comparando os valores previstos pelo modelo com os valores na base de treino, quanto mais próximo de 1, melhor:

In [ ]:
rfr.score(x_treinamento , y_treinamento)

- `.score()` - agora comparando os valores previstos pelo modelo com os valores que o modelo não usou, que estão na base teste, quanto mais próximo de 1.0, melhor:

In [ ]:
rfr.score(x_teste, y_teste)

- `mean_squared_error` - Raiz Quadrada do Erro Quadrático Médio: calcula uma medida da diferença entre os valores previstos e os valores reais, quanto mais próximo de 0, melhor:

In [ ]:
rmse = np.sqrt(mean_squared_error(y_teste, y_predicao))
rmse

- Histograma dos resíduos do modelo - visualizaçao gráfica da difereça entre os valores previstos e os valores reais:

In [ ]:
residuos = y_teste - y_predicao

plt.figure(figsize=(15, 7.5))
plt.hist(residuos,edgecolor='white', bins=len(y_teste));
plt.title('Histograma dos resíduos do modelo de Random Forest Regression')
plt.ylabel('Quantidade')
plt.xlabel('Valor dos resíduos')
plt.show()

### 2. Regressão Linear

#### Na Regressão Linear, as relações são modeladas com as predições de funções lineares, onde seus parâmetros serão calculados a parir da base de treinamento.

Para implementar o modelo será utilizado `LinearRegression` do módulo `sklearn.linear_model`.

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lr = LinearRegression()
lr.fit(x_treinamento, y_treinamento)
y_predicao = lr.predict(x_teste)

#### Verificando a performance:
- `.score()` (varia de 0 a 1) - comparando os valores previstos pelo modelo com os valores na base de treino, quanto mais próximo de 1, melhor:

In [ ]:
lr.score(x_treinamento , y_treinamento)

- `.score()` - agora comparando os valores previstos pelo modelo com os valores que o modelo não usou, que estão na base teste, quanto mais próximo de 1.0, melhor:

In [ ]:
lr.score(x_teste, y_teste)

- `mean_squared_error` - Raiz Quadrada do Erro Quadrático Médio: calcula uma medida da diferença entre os valores previstos e os valores reais, quanto mais próximo de 0, melhor:

In [ ]:
rmse = np.sqrt(mean_squared_error(y_teste, y_predicao))
rmse

- Histograma dos resíduos do modelo - visualizaçao gráfica da difereça entre os valores previstos e os valores reais:

In [ ]:
residuos = y_teste - y_predicao

plt.figure(figsize=(15, 7.5))
plt.hist(residuos,edgecolor='white', bins=len(y_teste));
plt.title('Histograma dos resíduos do modelo de Regressão Linear')
plt.ylabel('Quantidade')
plt.xlabel('Valor dos resíduos')
plt.show()

___

<div id="conclusao"></div>

## D. Conclusão

- A partir do tema e pergunta apresentados e uma análise exploratória inicial, foram escolhidas dois modelos de predição.
- De acordo com os tipos de variáveis trabalhadas, foram utilizadas duas técnicas de regressão.
- Os modelos de Random Forest Regression e regressão Linear foram implementados e calculados.
- Diferentes técnicas de validação e medição de performance foram apresentadas e implementadas.
- Utilizando as técnicas de validação, o modelo de Random Forest Regression foi o que apresentou a melhor performance.

___

<div id="refs"></div>

## E. Referências Bibliográficas

- BASE DE DADOS: https://www.kaggle.com/therealsampat/predict-movie-success-rate
- https://ensinandomaquinasblog.wordpress.com/2017/12/15/modelos-preditivos-de-notas-de-redacao-do-enem-2015/
- https://stackabuse.com/random-forest-algorithm-with-python-and-scikit-learn
- https://heartbeat.fritz.ai/random-forest-regression-in-python-using-scikit-learn-9e9b147e2153
- https://stackabuse.com/linear-regression-in-python-with-scikit-learn
- https://paulovasconcellos.com.br/como-saber-se-seu-modelo-de-machine-learning-est%C3%A1-funcionando-mesmo-a5892f6468b